In [0]:
%sql
use redox.main;

## Copy Files UDF

In [0]:
from pyspark.sql.functions import col, lit, concat
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, MapType

def copy_file(src_path, dest_path):
    try:
        with open(src_path, 'rb') as src_file, open(dest_path, 'wb') as dest_file:
            dest_file.write(src_file.read())
        return {"status": "success"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

copy_file_udf = udf(copy_file, MapType(StringType(), StringType()))

In [0]:
df = spark.table("ccda_bronze").limit(2)
# display(df)

In [0]:
copied_files = (
  df
  .withColumn("source_path", col("file_metadata.file_path"))
  .withColumn("destination_path",  concat(lit("/Volumes/redox/main/extract_staging/ccda/"), col("file_metadata.file_name")))
  .withColumn("sent_to_redox", copy_file_udf(col("source_path"), col("destination_path")))
)

In [0]:
display(copied_files)

***